In [1]:
from deepCNN import deepCNN
from utils import *
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [2]:
model = deepCNN()
print(model)

deepCNN(
  (ConvLayer1): Sequential(
    (0): Conv2d(1, 25, kernel_size=(1, 10), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): Conv2d(25, 25, kernel_size=(22, 1), stride=(1, 1))
    (3): ELU(alpha=1.0)
    (4): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  )
  (elu): ELU(alpha=1.0)
  (maxpool): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(25, 50, kernel_size=(1, 10), stride=(1, 1))
  (conv3): Conv2d(50, 100, kernel_size=(1, 10), stride=(1, 1))
  (conv4): Conv2d(100, 200, kernel_size=(1, 10), stride=(1, 1))
)


In [3]:
X_train, y_train, X_test, y_test = init_data(subject=None, verbose=True)

train_loader, val_loader = load_data(X_train, y_train, X_test, y_test, verbose=True)



print(train_loader)

In [4]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

print("Using", device)

Using mps


In [5]:
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
if torch.cuda.is_available():
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cpu')

print("Using", device)

Using mps


In [8]:
num_epochs = 10  # Number of epochs to train for

for epoch in tqdm(range(num_epochs)):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    train_count = 0
    train_correct_count = 0

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Compute accuracy
        with torch.no_grad():
            y_hat = torch.argmax(outputs, dim=-1)
            train_correct_count += torch.sum(y_hat == labels, axis=-1)
            train_count += inputs.size(0)
            
    train_acc = train_correct_count / train_count

    model.eval()

    val_count = 0
    val_correct_count = 0
    with torch.no_grad():
        for idx, (val_x, val_y) in enumerate(val_loader):
            val_x = val_x.float().to(device)
            val_y = val_y.long().to(device)
            logits = model(val_x).detach()
            y_hat = torch.argmax(logits, dim=-1)
            val_correct_count += torch.sum(y_hat == val_y, axis=-1)
            val_count += val_x.size(0)
    val_acc = val_correct_count / val_count

    print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

  0%|          | 0/10 [00:14<?, ?it/s]


RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same